# 실험 02: Chain of Thought (단계별 사고) 기법

**복잡한 추론 문제에서 AI의 정확도를 높이는 핵심 기법**

---

## 1. 개요

### 목적
**Chain of Thought (CoT)** 프롬프팅이 복잡한 수학/논리 문제에서 AI의 정확도를 얼마나 향상시키는지 검증합니다.

### 배경
사람도 복잡한 문제를 풀 때 **중간 과정**을 거칩니다:

```
문제: 철수는 사과 5개가 있었습니다. 2개를 주고, 3개를 받았습니다. 몇 개?

❌ 바로 답하기: "6개" (틀릴 수 있음)
✅ 단계별 사고: "5개 - 2개 = 3개, 3개 + 3개 = 6개" (검증 가능)
```

AI도 마찬가지입니다. 단계별로 생각하게 하면 정확도가 올라갑니다.

### 가설
1. CoT 프롬프트는 복잡한 문제에서 정확도를 **크게 향상**시킬 것이다
2. 단순한 문제에서는 효과가 **적을** 것이다
3. CoT는 토큰을 **더 많이 사용**하지만, 정확도 향상으로 상쇄될 것이다

## 2. Chain of Thought란?

### 정의
**Chain of Thought (CoT)**는 AI가 최종 답을 내기 전에 **중간 추론 과정**을 생성하도록 유도하는 프롬프팅 기법입니다.

### 작동 원리

```
[기존 방식]
Q: 8 + 7 × 2 = ?
A: 22 ← 바로 답

[CoT 방식]
Q: 8 + 7 × 2 = ?
A: 먼저 곱셈을 계산합니다: 7 × 2 = 14
   그 다음 덧셈을 합니다: 8 + 14 = 22
   따라서 답은 22입니다. ← 과정 + 답
```

### 학술적 근거

| 논문 | 발표 | 핵심 발견 |
|------|------|----------|
| "Chain-of-Thought Prompting Elicits Reasoning" | NeurIPS 2022 (Google) | GSM8K 수학 문제에서 **+40%** 정확도 향상 |
| "Self-Consistency Improves Chain of Thought" | ICLR 2023 | 여러 CoT 경로의 다수결로 추가 향상 |
| "Large Language Models are Zero-Shot Reasoners" | NeurIPS 2022 | "Let's think step by step" 한 줄로도 효과 |

### 왜 효과적인가?

1. **오류 조기 발견**: 중간 과정에서 실수를 잡을 수 있음
2. **복잡성 분해**: 어려운 문제를 작은 단계로 나눔
3. **맥락 활용**: 이전 단계의 결과를 다음 단계에서 참조

## 3. 환경 설정

### 사용 기술
| 항목 | 값 | 설명 |
|------|------|------|
| 모델 | qwen2.5:7b | Alibaba의 오픈소스 LLM |
| 프레임워크 | LangChain | LLM 애플리케이션 개발 |
| 평가 방식 | 정답 포함 여부 | CoT는 설명이 포함되므로 |

### 평가 지표
| 지표 | 설명 | 계산 방법 |
|------|------|----------|
| 정확도(Accuracy) | 정답을 맞춘 비율 | 정답 수 / 전체 문제 수 |
| 토큰 수 | 사용한 토큰 | 입력 + 출력 토큰 |
| 응답 시간 | 답변 생성 시간 | 초 단위 |

In [ ]:
# ============================================================
# 환경 설정 및 라이브러리 임포트
# ============================================================

from langchain_ollama import ChatOllama
import tiktoken
import time
import json
import re
from datetime import datetime

# LLM 초기화
# temperature=0: 결정적 출력으로 실험의 재현성 보장
llm = ChatOllama(model="qwen2.5:7b", temperature=0)

# 토큰 카운터
encoder = tiktoken.encoding_for_model("gpt-3.5-turbo")

print("환경 설정 완료")
print(f"- 모델: qwen2.5:7b")
print(f"- Temperature: 0")

In [ ]:
# ============================================================
# 평가 함수 정의
# ============================================================

def count_tokens(text: str) -> int:
    """텍스트의 토큰 수를 계산합니다."""
    return len(encoder.encode(text))


def extract_number(text: str) -> int:
    """
    응답에서 숫자를 추출합니다.
    
    CoT 응답은 "따라서 답은 6입니다"처럼 문장 형태이므로,
    마지막에 나온 숫자를 답으로 간주합니다.
    
    Args:
        text: AI의 응답 텍스트
    
    Returns:
        int: 추출된 숫자 (없으면 0)
    """
    # 모든 숫자를 찾아서 마지막 숫자 반환
    numbers = re.findall(r'\d+', text)
    return int(numbers[-1]) if numbers else 0


def check_answer(response: str, expected: str) -> bool:
    """
    응답이 정답을 포함하는지 확인합니다.
    
    CoT 응답은 "6입니다", "답: 6" 등 다양한 형태이므로,
    정답이 응답에 '포함'되어 있으면 정답으로 처리합니다.
    
    Args:
        response: AI의 응답
        expected: 예상 정답
    
    Returns:
        bool: 정답 여부
    """
    response_clean = response.strip().lower()
    expected_clean = str(expected).strip().lower()
    
    # 방법 1: 정답이 응답에 포함됨
    if expected_clean in response_clean:
        return True
    
    # 방법 2: 추출한 숫자가 정답과 일치
    extracted = extract_number(response)
    try:
        return extracted == int(expected)
    except ValueError:
        return False


print("평가 함수 정의 완료")

## 4. 실험 설계

### 테스트 문제
다단계 추론이 필요한 **수학 문제 10개**를 사용합니다.

| 난이도 | 문제 유형 | 개수 |
|--------|----------|------|
| 쉬움 | 2단계 연산 | 5개 |
| 어려움 | 3단계 이상 연산 | 5개 |

### 비교 대상
| 방식 | 프롬프트 특징 |
|------|-------------|
| Zero-shot | "답만 말해줘" |
| Chain of Thought | "단계별로 생각해봅시다" |

In [ ]:
# ============================================================
# 테스트 문제 정의
# ============================================================

# 수학 문제 (다단계 추론 필요)
test_problems = [
    # 쉬운 문제 (2단계)
    {"q": "철수는 사과 5개가 있었습니다. 영희에게 2개를 주고, 엄마에게 3개를 새로 받았습니다. 철수에게 사과가 몇 개 있나요?", "a": 6},
    {"q": "버스에 12명이 타고 있었습니다. 첫 번째 정류장에서 4명이 내리고, 두 번째 정류장에서 7명이 탔습니다. 버스에 몇 명이 있나요?", "a": 15},
    {"q": "사탕 봉지에 24개의 사탕이 있습니다. 4명의 아이에게 똑같이 나눠주면 한 명당 몇 개를 받나요?", "a": 6},
    {"q": "도서관에서 책 3권을 빌렸고, 이미 집에 5권이 있습니다. 2권을 읽고 반납했다면 지금 몇 권이 있나요?", "a": 6},
    {"q": "가게에서 1000원짜리 물건 2개와 500원짜리 물건 3개를 샀습니다. 총 얼마를 지불해야 하나요?", "a": 3500},
    
    # 어려운 문제 (3단계 이상)
    {"q": "과수원에 사과나무 15그루와 배나무 10그루가 있습니다. 각 사과나무에서 8개, 배나무에서 12개의 열매가 열렸습니다. 총 열매는 몇 개인가요?", "a": 240},
    {"q": "학교에 남학생 120명, 여학생 80명이 있습니다. 전체의 25%가 안경을 쓴다면 안경을 쓴 학생은 몇 명인가요?", "a": 50},
    {"q": "한 상자에 연필 12자루가 들어있습니다. 5상자를 사서 8명에게 똑같이 나눠주면 한 명당 몇 자루를 받나요?", "a": 7},
    {"q": "기차가 시속 80km로 달립니다. 2시간 30분 동안 달리면 몇 km를 이동하나요?", "a": 200},
    {"q": "직사각형의 가로가 15cm, 세로가 8cm입니다. 둘레의 길이는 몇 cm인가요?", "a": 46},
]

print(f"테스트 문제: {len(test_problems)}개")
print("\n샘플 문제:")
for i, p in enumerate(test_problems[:3], 1):
    print(f"  {i}. {p['q'][:40]}... (정답: {p['a']})")

## 5. 실험 실행

### 5.1 프롬프트 정의

두 가지 프롬프트를 비교합니다:

**Zero-shot (기본)**
```
문제: {문제}
답 (숫자만):
```

**Chain of Thought**
```
문제: {문제}
차근차근 단계별로 생각해봅시다.
풀이:
```

In [ ]:
# ============================================================
# 프롬프트 템플릿 정의
# ============================================================

# Zero-shot: 답만 요구
# - 단순하고 빠름
# - 복잡한 문제에서 실수 가능
ZERO_SHOT_TEMPLATE = """문제: {q}

답 (숫자만):"""

# Chain of Thought: 단계별 사고 유도
# - "차근차근 단계별로" 가 핵심 트리거
# - 중간 과정을 생성하게 함
COT_TEMPLATE = """문제: {q}

차근차근 단계별로 생각해봅시다.

풀이:"""

print("[Zero-shot 프롬프트 예시]")
print(ZERO_SHOT_TEMPLATE.format(q="5 + 3 = ?"))
print()
print("[CoT 프롬프트 예시]")
print(COT_TEMPLATE.format(q="5 + 3 = ?"))

In [ ]:
# ============================================================
# 실험 실행 함수
# ============================================================

def run_experiment(template: str, problems: list, name: str) -> dict:
    """
    주어진 프롬프트 템플릿으로 모든 문제를 풀고 결과를 집계합니다.
    
    Args:
        template: 프롬프트 템플릿 (문자열)
        problems: 테스트 문제 리스트
        name: 실험 이름
    
    Returns:
        dict: 정확도, 토큰 수, 시간 등 결과
    """
    results = []
    correct_count = 0
    total_tokens = 0
    total_time = 0
    
    print(f"\n[{name}] 실험 중...")
    
    for i, problem in enumerate(problems, 1):
        # 프롬프트 생성
        prompt = template.format(q=problem["q"])
        
        # 실행 및 시간 측정
        start = time.time()
        response = llm.invoke(prompt).content
        elapsed = time.time() - start
        
        # 토큰 계산
        tokens = count_tokens(prompt) + count_tokens(response)
        
        # 정답 확인
        is_correct = check_answer(response, problem["a"])
        
        # 집계
        if is_correct:
            correct_count += 1
        total_tokens += tokens
        total_time += elapsed
        
        # 진행 상황 출력
        status = "O" if is_correct else "X"
        print(f"  문제 {i:2d}: {status} (토큰: {tokens}, 시간: {elapsed:.2f}초)")
        
        # 상세 결과 저장
        results.append({
            "question": problem["q"][:50] + "...",
            "expected": problem["a"],
            "extracted": extract_number(response),
            "correct": is_correct,
            "tokens": tokens,
            "time": round(elapsed, 2)
        })
    
    # 결과 요약
    n = len(problems)
    return {
        "name": name,
        "accuracy": correct_count / n,
        "correct_count": correct_count,
        "total": n,
        "avg_tokens": total_tokens / n,
        "avg_time": total_time / n,
        "details": results
    }

In [ ]:
# ============================================================
# 실험 실행: Zero-shot vs Chain of Thought
# ============================================================

print("=" * 60)
print("실험: Zero-shot vs Chain of Thought")
print("=" * 60)

# 두 방식으로 실험 실행
zero_shot_results = run_experiment(ZERO_SHOT_TEMPLATE, test_problems, "Zero-shot")
cot_results = run_experiment(COT_TEMPLATE, test_problems, "Chain of Thought")

print("\n실험 완료!")

## 6. 결과 분석

### 6.1 정량적 결과

In [ ]:
# ============================================================
# 결과 비교 및 분석
# ============================================================

print("=" * 60)
print("실험 결과 비교")
print("=" * 60)

# 정확도 비교
zs_acc = zero_shot_results['accuracy'] * 100
cot_acc = cot_results['accuracy'] * 100
acc_improvement = cot_acc - zs_acc

# 토큰 비교
zs_tokens = zero_shot_results['avg_tokens']
cot_tokens = cot_results['avg_tokens']
token_increase = ((cot_tokens - zs_tokens) / zs_tokens) * 100

# 시간 비교
zs_time = zero_shot_results['avg_time']
cot_time = cot_results['avg_time']
time_increase = ((cot_time - zs_time) / zs_time) * 100 if zs_time > 0 else 0

print("\n[성능 비교표]")
print(f"┌{'─'*18}┬{'─'*15}┬{'─'*15}┬{'─'*15}┐")
print(f"│ {'지표':<16} │ {'Zero-shot':<13} │ {'CoT':<13} │ {'변화':<13} │")
print(f"├{'─'*18}┼{'─'*15}┼{'─'*15}┼{'─'*15}┤")
print(f"│ {'정확도':<16} │ {zs_acc:>10.1f}%  │ {cot_acc:>10.1f}%  │ {acc_improvement:>+10.1f}%p │")
print(f"│ {'평균 토큰':<16} │ {zs_tokens:>11.0f}  │ {cot_tokens:>11.0f}  │ {token_increase:>+10.1f}% │")
print(f"│ {'평균 시간(초)':<16} │ {zs_time:>11.2f}  │ {cot_time:>11.2f}  │ {time_increase:>+10.1f}% │")
print(f"└{'─'*18}┴{'─'*15}┴{'─'*15}┴{'─'*15}┘")

print(f"\n[핵심 발견]")
print(f"  - CoT 정확도 향상: {acc_improvement:+.1f}%p")
print(f"  - 토큰 사용 증가: {token_increase:+.1f}%")
print(f"  - 정확도 1%p 향상당 토큰 증가: {token_increase/acc_improvement:.1f}% (효율성 지표)" if acc_improvement > 0 else "  - 정확도 향상 없음")

In [ ]:
# ============================================================
# 문제별 상세 결과
# ============================================================

print("\n[문제별 상세 결과]")
print(f"{'문제':<45} {'정답':<8} {'Zero-shot':<12} {'CoT':<12}")
print("-" * 80)

for i, (zs, cot) in enumerate(zip(zero_shot_results['details'], cot_results['details'])):
    zs_status = "O" if zs['correct'] else "X"
    cot_status = "O" if cot['correct'] else "X"
    print(f"{zs['question'][:43]:<45} {zs['expected']:<8} {zs_status} ({zs['extracted']})     {cot_status} ({cot['extracted']})")

### 6.2 정성적 분석

**Zero-shot의 문제점**
- 복잡한 다단계 연산에서 중간 과정 생략으로 오류 발생
- 문제를 충분히 분석하지 않고 바로 답을 도출

**CoT의 장점**
- 각 단계를 명시적으로 수행하여 오류 감소
- 중간 결과를 확인하며 검증 가능
- 어려운 문제일수록 효과가 큼

**CoT의 단점**
- 토큰 사용량 증가 (비용 상승)
- 응답 시간 증가
- 단순한 문제에서는 오버헤드만 발생

## 7. 결론

### 핵심 발견

1. **CoT는 복잡한 추론 문제에서 효과적**
   - 다단계 수학 문제에서 정확도 향상
   - "단계별로 생각하자"는 트리거 문구가 핵심

2. **토큰 효율성 trade-off**
   - 정확도는 올라가지만 토큰 사용량도 증가
   - 정확도가 중요한 작업에서는 CoT가 유리

3. **문제 복잡도에 따른 선택**
   - 단순 문제 → Zero-shot (효율적)
   - 복잡 문제 → CoT (정확도 우선)

### 실무 적용 가이드

| 상황 | 권장 방식 | 이유 |
|------|----------|------|
| 단순 계산/분류 | Zero-shot | 빠르고 저렴 |
| 다단계 수학 문제 | CoT | 정확도 중요 |
| 논리 추론 | CoT | 중간 과정 필요 |
| 코드 디버깅 | CoT | 단계별 분석 필요 |

### 한계점
- 테스트 문제가 10개로 제한적
- 한국어 문제만 테스트
- 단일 모델(qwen2.5:7b)만 사용

### 다음 실험
- Few-shot Learning: 예시를 제공하면 성능이 더 올라갈까?
- Self-Consistency: 여러 번 풀어서 다수결로 결정하면?

In [ ]:
# ============================================================
# 결과 저장
# ============================================================

experiment_results = {
    "experiment": "02_Chain_of_Thought",
    "date": datetime.now().isoformat(),
    "model": "qwen2.5:7b",
    "hypothesis": "CoT가 복잡한 추론 문제에서 정확도를 향상시킨다",
    "test_problems_count": len(test_problems),
    "results": {
        "zero_shot": {
            "accuracy": f"{zs_acc:.1f}%",
            "correct": zero_shot_results['correct_count'],
            "total": zero_shot_results['total'],
            "avg_tokens": round(zs_tokens),
            "avg_time": f"{zs_time:.2f}s"
        },
        "chain_of_thought": {
            "accuracy": f"{cot_acc:.1f}%",
            "correct": cot_results['correct_count'],
            "total": cot_results['total'],
            "avg_tokens": round(cot_tokens),
            "avg_time": f"{cot_time:.2f}s"
        }
    },
    "comparison": {
        "accuracy_improvement": f"{acc_improvement:+.1f}%p",
        "token_increase": f"{token_increase:+.1f}%",
        "time_increase": f"{time_increase:+.1f}%"
    },
    "conclusion": {
        "finding": "CoT는 복잡한 추론 문제에서 정확도를 향상시킨다",
        "tradeoff": "정확도 향상 vs 토큰 사용량 증가",
        "recommendation": "복잡한 문제에서 CoT 사용 권장"
    },
    "academic_reference": {
        "paper": "Chain-of-Thought Prompting Elicits Reasoning in Large Language Models",
        "venue": "NeurIPS 2022",
        "authors": "Wei et al. (Google Research)"
    }
}

# JSON 파일로 저장
with open("../results/02_cot_results.json", "w", encoding="utf-8") as f:
    json.dump(experiment_results, f, ensure_ascii=False, indent=2)

print("실험 결과가 저장되었습니다: results/02_cot_results.json")
print("\n" + "=" * 60)
print("실험 완료!")
print("=" * 60)

---

## 참고 자료

1. Wei, J., et al. (2022). "Chain-of-Thought Prompting Elicits Reasoning in Large Language Models." NeurIPS 2022.
2. Wang, X., et al. (2023). "Self-Consistency Improves Chain of Thought Reasoning in Language Models." ICLR 2023.
3. Kojima, T., et al. (2022). "Large Language Models are Zero-Shot Reasoners." NeurIPS 2022.

---

*이 실험은 프롬프트 엔지니어링 포트폴리오 프로젝트의 일부입니다.*